In [56]:
import pandas as pd
import numpy as np
import re

In [20]:
def preprocess(row):
    row = re.sub('ㅁ', "", row)
    row = re.sub(r"\n", ",", row)
    
    # 용어
    row = re.sub("YoY|yoy|YOY", "전년 대비 증감율", row)
    row = re.sub("QoQ|qoq|QOQ", "전분기 대비 증감율", row)
    row = re.sub("MoM|mom|MOM", "전월 대비 증감율", row)
    row = re.sub("YTD", "연초 누계", row)
    row = re.sub("MTD", "월초 누계", row)
    
    # 분기 표현 1Q21->21년도 1분기
    quarter_rule = re.compile(r'([1-4]Q[\d]{2})')
    quarter_text = quarter_rule.finditer(row)
    startIdxList = []
    for text in quarter_text :
        startIdxList.append(text.start())
    for i in range(len(startIdxList)-1, -1, -1) :
        idx = startIdxList[i]
        quar = row[idx:idx+4]
        new_quar_text = row[idx+2:idx+4]+"년도 "+row[idx]+"분기 "
        row = row.replace(quar, new_quar_text)
        
    # [내용] 제거
    row = re.sub(r'\[[^)]*\]', '', row)
        
    # 다중 띄어쓰기
    row = re.sub(' +', ' ', row)
    
    # 앞뒤 공백
    row = re.sub(r"^\s+", "", row, flags=re.UNICODE)
    row = re.sub(r"\s+$", "", row, flags=re.UNICODE)
    
    return row

In [62]:
filename = "./combined_dataset.xlsx"
# filename = "./report.xlsx"
df = pd.read_excel(filename)

# print(df.isnull().sum())
# print(df)

logCnt = 1000
plusCnt = 1000
for i in range(len(df)) :
    if(i == logCnt) :
        print(i)
        logCnt += plusCnt
#     print(i, df['title'][i])
    
    # 영어로만 구성된 row는 drop
    if(len(re.findall(u'[가-힣]+', df['title'][i]))>0) :
        df['title'][i] = preprocess(df['title'][i])
    else :
        df = df.drop(i)

df['title'].replace('', np.nan, inplace=True)
df = df.dropna(subset=['title'])
df = df.reset_index(drop=True)
print("Done!")
# print(df[250:260])

C:\Users\user\AppData\Local\Temp/ipykernel_10020/3842574801.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'][i] = preprocess(df['title'][i])


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
date     0
title    0
dtype: int64
date     0
title    3
dtype: int64
Done!
          date                                              title
250 2021-11-18  "알리페이 지분에" 코스피200 편입 제동 걸린 카카오페이…'국민주' 등극한 카카오...
251 2021-11-19              ‘7만 전자’ 박스권 갇힌 삼성, 미·중 반도체 패권 전쟁 수혜보나
252 2021-11-21                 ‘국민주’ 삼전·카카오 부침에 매도하는 개미들, 사들이는 기관
253 2021-11-22                   삼전 급반등에 반도체 부품株 동반 강세…원익IPS 12%↑
254 2021-11-22                반도체 '투톱' 5~7%대 급등하니... 코스피 3000선 탈환
255 2021-11-22                           반도체 '투톱' 쌍끌이…코스피 3010 회복
256 2021-11-22                                삼전·하이닉스 개미 간만에 ‘활짝’
257 2021-11-22                    외국인 국내주식 망한 이유…계란을 삼전 바구니에만 담아서
258 2021-11-22                    삼성전자·SK하이닉스 주가 고공행진…D램시장 바닥 찍었나
259 2021-11-22                       반도체 겨울 끝났나…삼전 5%↑ SK하이닉스 7%↑


In [63]:
df.to_excel("./combined_dataset_cleaned.xlsx", index = False)